In [ ]:
%config InlineBackend.figure_format = 'retina'

import os
import sys
import numpy as np
import matplotlib.pyplot as plt

if sys.platform == "emscripten":
    import piplite

    await piplite.install("miepython", deps=False)
    os.environ["MIEPYTHON_USE_JIT"] = "0"  # jupyterlite cannot use numba

import miepython as mie
from miepython.field import e_near, e_plane, e_far
from miepython.vsh import mie_pi, mie_tau, M_odd, N_even
from miepython.util import cs, phasor_str

In [ ]:
def e_near_far(lambda0, d_sphere, m_index, r, theta, phi):
    """
    Compare the electric field in the far field for near and far calculations.

    Args:
        lambda0 (float): Wavelength the incident wave in vacuum.
        d_sphere (float): Diameter of the sphere.
        m (complex): Rrefractive index at position r
        r (float): Radial distance at which the field is evaluated.
        theta (float): Scattering angle (from z-axis) in radians.
        phi (float): Azimuthal angle (from x-axis) in radians.
        norm: type of normalization to use for scattering function

    Returns:
        tuple: Electric field components (E_r, E_theta, E_phi).
    """
    x = np.pi * d_sphere / lambda0
    jkr = 1j * 2 * np.pi * r / lambda0
    mu = np.cos(theta)
    a, b = mie.an_bn(m_index, x, 0)
    N = len(a)
    pi = np.zeros(N)
    tau = np.zeros(N)
    n = np.arange(1, N + 1)
    scale = (2 * n + 1) / ((n + 1) * n)
    scale2 = 1j**n * (2 * n + 1) / ((n + 1) * n)
    amp = np.exp(jkr) / (-jkr)

    complex(0)
    E_theta = complex(0)
    E_phi = complex(0)

    P_r = complex(0)
    P_theta = complex(0)
    P_phi = complex(0)

    mie._pi_tau(mu, pi, tau)

    for k in range(N):
        Mn = M_odd(k + 1, lambda0, d_sphere, m_index, r, theta, phi)
        Nn = N_even(k + 1, lambda0, d_sphere, m_index, r, theta, phi)

        P_r += scale2[k] * (1j * a[k] * Nn[0] - b[k] * Mn[0])
        th_part = scale2[k] * (1j * a[k] * Nn[1] - b[k] * Mn[1])
        P_theta += th_part
        print(k + 1, cs(th_part, -5), cs(P_theta, -5))
        P_phi += scale2[k] * (1j * a[k] * Nn[2] - b[k] * Mn[2])

        th_part = scale[k] * (tau[k] * a[k] + pi[k] * b[k]) * amp * np.cos(phi)
        E_theta += th_part
        print(k + 1, cs(th_part, -5), cs(E_theta, -5))
        E_phi += scale[k] * (pi[k] * a[k] + tau[k] * b[k]) * amp * np.sin(phi)
        print()

In [ ]:
d_sphere = 5
lambda0 = 1
n_sphere = 1.5
n_env = 1
theta = np.radians(45)
phi = np.radians(45)

k = 2 * np.pi / lambda0
x = np.pi * d_sphere / lambda0
r = 1000

e_near_far(lambda0, d_sphere, n_env, r, theta, phi)

In [ ]:
d_sphere = 5
lambda0 = 1
n_sphere = 1.5
n_env = 1
theta = np.radians(45)
phi = np.radians(45)

k = 2 * np.pi / lambda0
x = np.pi * d_sphere / lambda0
r = 1000

qext, qsca, qback, g = mie.efficiencies_mx(n_sphere / n_env, x)

a, b, c, d = mie.coefficients(n_sphere / n_env, x, internal=True)
abcd = np.array([a, b, c, d])

Er, Etheta, Ephi = e_near(abcd, lambda0, d_sphere, n_env, r, theta, phi)
print()

for norm in ["wiscombe"]:
    Fr, Ftheta, Fphi = e_far(lambda0, d_sphere, n_sphere, r, theta, phi)[:, 0]
    print("\nNORM = ", norm)
    print("𝜃 abs ratio %10.7f or %10.7f" % (abs(Etheta) / abs(Ftheta), abs(Ftheta) / abs(Etheta)))
    print("ɸ abs ratio %10.7f or %10.7f" % (abs(Ephi) / abs(Fphi), abs(Fphi) / abs(Ephi)))
    print()
    print("       Etheta               Ephi")
    print(cs(Etheta), " |", cs(Ephi))
    print(cs(Ftheta), " |", cs(Fphi))
    print("                       |")
    print(phasor_str(Etheta, 10), "|", phasor_str(Ephi, 10))
    print(phasor_str(Ftheta, 10), "|", phasor_str(Fphi, 10))

In [ ]:
d_sphere = 1
lambda0 = 1
n_sphere = 1.5
n_env = 1
theta = np.pi / 2
phi = np.pi / 3

k = 2 * np.pi / lambda0
x = np.pi * d_sphere / lambda0

rr = np.linspace(0, d_sphere, 202)
nn = np.array([n_sphere if r < d_sphere / 2 else n_env for r in rr])
mm = nn / n_env

a, b, c, d = mie.coefficients(n_sphere / n_env, x, internal=True)
abcd = np.array([a, b, c, d])

E_r = np.array([])
E_theta = np.array([])
E_phi = np.array([])

for r, m in zip(rr, mm):
    Er, Etheta, Ephi = e_near(abcd, lambda0, d_sphere, n_env, r, theta, phi)
    E_r = np.append(E_r, Er)
    E_theta = np.append(E_theta, Etheta)
    E_phi = np.append(E_phi, Ephi)

print(abs(E_r[100]) / abs(E_r[101]), abs(E_theta[100]) / abs(E_theta[101]), abs(E_phi[100]) / abs(E_phi[101]))
plt.plot(rr, E_r * mm**2, "bo", markersize=3, label=r"$E_r$")
plt.plot(rr, E_theta, "rs", markersize=3, label=r"$E_\theta$")
plt.plot(rr, E_phi, "go", markersize=3, label=r"$E_\phi$")
plt.xlabel("radius [microns]")
plt.ylabel("Field Components")
plt.legend()
plt.show()

In [ ]:
d_sphere = 1
lambda0 = 1
n_sphere = 1.5
n_env = 1
thetas = np.linspace(0, np.pi, 100)
phi = np.pi / 3

k = 2 * np.pi / lambda0
x = np.pi * d_sphere / lambda0

rr = np.array([0.499, 0.501])
nn = np.array([n_sphere if r < d_sphere / 2 else n_env for r in rr])
mm = nn / n_env

a, b, c, d = mie.coefficients(n_sphere / n_env, x, internal=True)
abcd = np.array([a, b, c, d])

E_r = np.array([])
E_theta = np.array([])
E_phi = np.array([])

for theta in thetas:
    Er, Etheta, Ephi = e_near(abcd, lambda0, d_sphere, n_env, rr[0], theta, phi)
    Er1, Etheta1, Ephi1 = e_near(abcd, lambda0, d_sphere, n_env, rr[1], theta, phi)
    E_r = np.append(E_r, abs(Er) / abs(Er1))
    E_theta = np.append(E_theta, abs(Etheta) / abs(Etheta1))
    E_phi = np.append(E_phi, abs(Ephi) / abs(Ephi1))

plt.plot(np.degrees(thetas), E_r, "bo", markersize=3, label=r"$E_r$")
plt.plot(np.degrees(thetas), E_theta, "rs", markersize=3, label=r"$E_\theta$")
plt.plot(np.degrees(thetas), E_phi, "go", markersize=3, label=r"$E_\phi$")
plt.xlabel("angle [degrees]")
plt.ylabel("Field Components")
plt.legend()
plt.show()

In [ ]:
d_sphere = 1
lambda0 = 1
n_sphere = 1.5
n_env = 1
thetas = np.linspace(0, np.pi, 100)
phi = np.pi / 3

k = 2 * np.pi / lambda0
x = np.pi * d_sphere / lambda0

rr = np.array([0.499, 0.501])
nn = np.array([n_sphere if r < d_sphere / 2 else n_env for r in rr])
mm = nn / n_env

a, b, c, d = mie.coefficients(n_sphere / n_env, x, internal=True)
abcd = np.array([a, b, c, d])

E_r = np.array([])
E_theta = np.array([])
E_phi = np.array([])

for theta in thetas:
    Er, Etheta, Ephi = e_near(abcd, lambda0, d_sphere, n_env, rr[0], theta, phi)
    Er1, Etheta1, Ephi1 = e_near(abcd, lambda0, d_sphere, n_env, rr[1], theta, phi)
    E_r = np.append(E_r, abs(Er) / abs(Er1))
    E_theta = np.append(E_theta, abs(Etheta) / abs(Etheta1))
    E_phi = np.append(E_phi, abs(Ephi) / abs(Ephi1))

plt.plot(np.degrees(thetas), E_r, "bo", markersize=3, label=r"$E_r$")
plt.plot(np.degrees(thetas), E_theta, "rs", markersize=3, label=r"$E_\theta$")
plt.plot(np.degrees(thetas), E_phi, "go", markersize=3, label=r"$E_\phi$")
plt.xlabel("angle [degrees]")
plt.ylabel("Field Components")
plt.legend()
plt.show()

In [ ]:
N = 100
NN = 3000
xx = np.linspace(0, 15, N)
yy = np.linspace(0, 5, N)
zz = np.linspace(0, 5, N)

x = 0.5
y = 0.3
z = 0.3

Ex = np.zeros(N, dtype=complex)
Ey = np.zeros(N, dtype=complex)
Ez = np.zeros(N, dtype=complex)

for i, z in enumerate(zz):
    Ex[i], Ey[i], Ez[i] = e_plane(x, y, z, NN)

# plt.plot(xx, 5/(0.001+np.sqrt(xx*2+y*y+z*z)))
plt.plot(zz, abs(Ex))
# plt.plot(zz, abs(Ey))
# plt.plot(zz, abs(Ez))
# plt.ylim(0,1)
plt.grid(True)
plt.show()

In [ ]:
print(abs(0.5588004397152354 - 0.17127565209158688j) / n_sphere)
print(abs(0.2972637476495329 + 0.36749727738599286j))

In [ ]:
d_sphere = 1
lambda0 = 1
n_sphere = 1.5
n_env = 1
theta = np.pi / 3
phi = np.pi / 3


x = np.pi * d_sphere / lambda0
m = n_sphere / n_env
k = 2 * np.pi / lambda0

a, b, c, d = mie.coefficients(m, x, internal=True)
abcd = np.array([a, b, c, d])

rr = np.linspace(0, 1, 100)

E_r = np.array([])
E_theta = np.array([])
E_phi = np.array([])
for r in rr:
    Er, Etheta, Ephi = e_field(abcd, d_sphere, r, theta, phi, k)
    E_r = np.append(E_r, abs(Er))
    E_theta = np.append(E_theta, abs(Etheta))
    E_phi = np.append(E_phi, abs(Ephi))

plt.plot(rr, E_r, "bo", markersize=3, label=r"$E_r$")
plt.plot(rr, E_theta, "rs", markersize=3, label=r"$E_\theta$")
plt.plot(rr, E_phi, "go", markersize=3, label=r"$E_\phi$")
plt.xlabel("radius [microns]")
plt.ylabel("Field Components")
plt.legend()
plt.show()

In [ ]:
def pi_tau(mu, pi, tau):
    n_terms = len(pi)
    pi_nm2 = 0
    pi[0] = 1
    for n in range(1, n_terms):
        tau[n - 1] = n * mu * pi[n - 1] - (n + 1) * pi_nm2
        temp = pi[n - 1]
        pi[n] = ((2 * n + 1) * mu * temp - (n + 1) * pi_nm2) / n
        pi_nm2 = temp


N = 15
theta = np.pi / 5
mu = np.cos(theta)
pi = np.zeros(N)
tau = np.zeros(N)

pi_tau(mu, pi, tau)

for i in range(1, N):
    pp = mie_pi(i, theta)
    tt = mie_tau(i, theta)
    print("%2d  % 9.5f  % 9.5f  |  % 9.5f   % 9.5f" % (i, pp, pi[i - 1], tt, tau[i - 1]))

In [ ]:
%%timeit
# old way
mu = np.linspace(0, 1, 1000)
theta = np.arccos(mu)
N = 20

for j, th in enumerate(theta):
    for i in range(1, N):
        pp = mie_pi(i, th)
        tt = mie_tau(i, th)

In [ ]:
%%timeit
# new way
mu = np.linspace(0, 1, 1000)
theta = np.arccos(mu)
N = 20
pi = np.zeros(N)
tau = np.zeros(N)
for j, mm in enumerate(mu):
    pi_tau(mm, pi, tau)

In [ ]:
def x_S1_S2(m, x, mu, n_pole=0):
    """
    Calculate the scattering amplitude functions for spheres.

    The amplitude functions have been normalized so that when integrated
    over all 4*pi solid angles, the integral will be qext*pi*x**2.

    The units are weird, sr**(-0.5)

    Args:
        m: the complex index of refraction of the sphere
        x: the size parameter of the sphere
        mu: array of angles, cos(theta), to calculate scattering amplitudes
        norm_int: integer describing type of normalization
        n_pole: return n_pole term from series (default=0 means include all terms)
        e_field: If True then Electric field (does not currently work)

    Returns:
        S1, S2: the scattering amplitudes at each angle mu [sr**(-0.5)]
    """
    a, b = mie._an_bn(m, x, 0)

    nangles = len(mu)
    S1 = np.zeros(nangles, dtype=np.complex128)
    S2 = np.zeros(nangles, dtype=np.complex128)

    nstop = len(a)
    for k in range(nangles):
        pi_nm2 = 0
        pi_nm1 = 1
        for n in range(1, nstop):
            tau_nm1 = n * mu[k] * pi_nm1 - (n + 1) * pi_nm2
            if n_pole in (0, n):
                S1[k] += (2 * n + 1) * (pi_nm1 * a[n - 1] + tau_nm1 * b[n - 1]) / (n + 1) / n
                print(
                    "expected %3d % 8.3f % 8.3f % 8.3f %s %s"
                    % (n, (2 * n + 1) / (n + 1) / n, pi_nm1, tau_nm1, cs(a[n - 1]), cs(b[n - 1]))
                )
                S2[k] += (2 * n + 1) * (tau_nm1 * a[n - 1] + pi_nm1 * b[n - 1]) / (n + 1) / n
            temp = pi_nm1
            pi_nm1 = ((2 * n + 1) * mu[k] * pi_nm1 - (n + 1) * pi_nm2) / n
            pi_nm2 = temp

    return [np.conjugate(S1), np.conjugate(S2)]


def my_S1_S2(m, x, mu, n_pole):
    """
    Calculate the scattering amplitude functions for spheres.

    The amplitude functions have been normalized so that when integrated
    over all 4*pi solid angles, the integral will be qext*pi*x**2.

    The units are weird, sr**(-0.5)

    Args:
        m: the complex index of refraction of the sphere
        x: the size parameter of the sphere
        mu: array of angles, cos(theta), to calculate scattering amplitudes
        norm_int: integer describing type of normalization
        n_pole: return n_pole term from series (default=0 means include all terms)
        e_field: If True then Electric field (does not currently work)

    Returns:
        S1, S2: the scattering amplitudes at each angle mu [sr**(-0.5)]
    """
    a, b = mie._an_bn(m, x, 0)
    N = len(a)
    pi = np.zeros(N)
    tau = np.zeros(N)

    n = np.arange(1, N + 1)
    scale = (2 * n + 1) / ((n + 1) * n)

    nangles = len(mu)
    S1 = np.zeros(nangles, dtype=np.complex128)
    S2 = np.zeros(nangles, dtype=np.complex128)

    for k in range(nangles):
        pi_tau(mu[k], pi, tau)
        for i in range(N):
            print("actual   %3d % 8.3f % 8.3f % 8.3f %s %s" % (i, scale[i], pi[i], tau[i], cs(a[i]), cs(b[i])))

        S1[k] = np.sum(scale * (pi * a + tau * b))
        S2[k] = np.sum(scale * (tau * a + pi * b))

    return [np.conjugate(S1), np.conjugate(S2)]


mu = np.linspace(0.5, 0.5, 1)
S1m, S2m = my_S1_S2(1.5 - 1j, 3, mu, 0)
S1, S2 = mie._S1_S2(1.5 - 1j, 3, mu, 0)
S1x, S2x = x_S1_S2(1.5 - 1j, 3, mu)

for i, mm in enumerate(mu):
    print("%2d  %s  %s  %s |  %s   %s  %s" % (i, cs(S1[i]), cs(S1m[i]), cs(S1x[i]), cs(S2[i]), cs(S2m[i]), cs(S2x[i])))

In [ ]:
N = 10
n = np.arange(1, N + 1)
scale = (1j) ** n * (2 * n + 1) / n / (n + 1)
print(cs(scale))